### Rede Neural

In [3]:
import time
import numpy as np
from random import randint
from typing import List, Callable

class NN:
    """
    Uma classe para representar uma rede neural simples com uma camada oculta.
    """
    def __init__(self, inputs: int, hidden_units: int, outputs: int, activation: str = 'sigmoid'):
        """
        Inicializa a rede neural com pesos e biases aleatórios.

        Args:
            inputs (int): Número de neurônios na camada de entrada.
            hidden_units (int): Número de neurônios na camada oculta.
            outputs (int): Número de neurônios na camada de saída.
            activation (str): Nome da função de ativação ('sigmoid', 'tanh', 'relu').
        """
        # Arquitetura da rede
        self.inputs = inputs
        self.hidden_units = hidden_units
        self.outputs = outputs
        self.activation_str = activation

        # Parâmetros da rede (pesos e biases)
        # Camada 1: Entrada -> Oculta
        self.W1 = np.random.randn(hidden_units, inputs)  # Pesos
        self.b1 = np.random.randn(1, hidden_units)            # Biases

        # Camada 2: Oculta -> Saída
        self.W2 = np.random.randn(outputs, hidden_units) # Pesos
        self.b2 = np.random.randn(1, outputs)                 # Biases
        
        # Função de ativação
        self.actv = self._get_activation_function(activation)

    def _get_activation_function(self, name: str) -> Callable[[np.ndarray], np.ndarray]:
        """Retorna a função de ativação correspondente ao nome."""
        match name:
            case 'sigmoid':
                return lambda x: 1 / (1 + np.exp(-x))
            case 'tanh':
                return np.tanh
            case 'relu':
                return lambda x: np.maximum(0, x)
            case _:
                raise ValueError(f"Função de ativação '{name}' não suportada.")

    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Realiza a passagem para a frente (forward pass) na rede para fazer uma predição.
        """
        # Camada oculta
        z1 = X @ self.W1.T + self.b1
        a1 = self.actv(z1)
        
        # Camada de saída
        z2 = a1 @ self.W2.T + self.b2
        # A camada de saída não terá ativação neste exemplo (comum para regressão)
        output = z2
        
        return output

    def __repr__(self):
        """Representação em string do objeto NN."""
        return (f"NN(inputs={self.inputs}, hidden_units={self.hidden_units}, "
                f"outputs={self.outputs}, activation='{self.activation_str}')")


# --- 2. Funções para Achatar e Reconstruir os Parâmetros ---

def flatten_parameters(nn: NN) -> np.ndarray:
    """
    Pega uma rede neural (NN) e coloca todos os seus parâmetros (pesos e biases)
    alinhados em um único array numpy.

    Args:
        nn (NN): A rede neural a ser "achatada".

    Returns:
        np.ndarray: Um array 1D contendo todos os parâmetros da rede.
    """
    # Coleta todos os arrays de parâmetros
    params = [nn.W1, nn.b1, nn.W2, nn.b2]
    
    # Usa np.concatenate com uma lista de parâmetros achatados (.ravel())
    flat_params = np.concatenate([p.ravel() for p in params])
    
    return flat_params

def unflatten_parameters(nn: NN, flat_params: np.ndarray):
    """
    Pega um array 1D de parâmetros e os usa para redefinir os pesos e biases
    de uma rede neural (NN) existente.

    Args:
        nn (NN): A rede neural cujos parâmetros serão atualizados.
        flat_params (np.ndarray): O array 1D com os novos parâmetros.
    """
    # Calcula o número de elementos em cada matriz de parâmetros
    s_w1 = nn.W1.size
    s_b1 = nn.b1.size
    s_w2 = nn.W2.size
    s_b2 = nn.b2.size
    
    # Verifica se o tamanho do array de entrada corresponde ao total de parâmetros
    total_params = s_w1 + s_b1 + s_w2 + s_b2
    if flat_params.size != total_params:
        raise ValueError(f"Tamanho do array incorreto. Esperado: {total_params}, Recebido: {flat_params.size}")

    # Ponteiro para marcar a posição atual no array achatado
    pointer = 0
    
    # Extrai, remodela e atribui os parâmetros para a Camada 1
    nn.W1 = flat_params[pointer : pointer + s_w1].reshape(nn.W1.shape)
    pointer += s_w1
    nn.b1 = flat_params[pointer : pointer + s_b1].reshape(nn.b1.shape)
    pointer += s_b1
    
    # Extrai, remodela e atribui os parâmetros para a Camada 2
    nn.W2 = flat_params[pointer : pointer + s_w2].reshape(nn.W2.shape)
    pointer += s_w2
    nn.b2 = flat_params[pointer : pointer + s_b2].reshape(nn.b2.shape)

#### Teste NN

In [4]:
if __name__ == "__main__":
    # Definição da arquitetura da rede
    INPUTS = 2
    HIDDEN_UNITS = 4
    OUTPUT = 1
    ACTIVATION = 'sigmoid'

    # 1. Criar a primeira rede neural (nn1)
    print("--- Criando a Rede Neural 1 (nn1) ---")
    nn1 = NN(inputs=INPUTS, hidden_units=HIDDEN_UNITS, outputs=OUTPUT, activation=ACTIVATION)
    print(f"Rede criada: {nn1}")
    
    # Criar um dado de entrada de exemplo
    input_data = np.random.normal(0, 1, (1, INPUTS))
    print(f"\nEntrada de exemplo (shape {input_data.shape}):\n{input_data}")

    # Fazer uma predição com a nn1
    output1 = nn1.predict(input_data)
    print(f"\nSaída da nn1 (com pesos originais): {output1.item():.4f}")

    # 2. Achatando os parâmetros da nn1
    print("\n--- Achatando parâmetros da nn1 ---")
    flat_array = flatten_parameters(nn1)
    print(f"Array de parâmetros achatado (shape {flat_array.shape}):")
    print(flat_array)

    # 3. Criar uma segunda rede neural (nn2), com pesos e biases diferentes
    print("\n--- Criando a Rede Neural 2 (nn2) com pesos diferentes ---")
    nn2 = NN(inputs=INPUTS, hidden_units=HIDDEN_UNITS, outputs=OUTPUT, activation=ACTIVATION)
    
    # Fazer uma predição com a nn2 para mostrar que o resultado é diferente
    output2_antes = nn2.predict(input_data)
    print(f"Saída da nn2 (antes da cópia): {output2_antes.item():.4f}")

    # 4. Reconstruir a nn2 usando os parâmetros achatados da nn1
    print("\n--- Reconstruindo nn2 com os parâmetros da nn1 ---")
    unflatten_parameters(nn2, flat_array)
    print("Parâmetros de nn1 foram copiados para nn2.")

    # 5. Verificar se as saídas agora são idênticas
    print("\n--- Verificando os resultados ---")
    output2_depois = nn2.predict(input_data)
    print(f"Saída da nn1 original:          {output1.item():.8f}")
    print(f"Saída da nn2 após reconstrução: {output2_depois.item():.8f}")

--- Criando a Rede Neural 1 (nn1) ---
Rede criada: NN(inputs=2, hidden_units=4, outputs=1, activation='sigmoid')

Entrada de exemplo (shape (1, 2)):
[[1.76530781 1.05662411]]

Saída da nn1 (com pesos originais): -1.8337

--- Achatando parâmetros da nn1 ---
Array de parâmetros achatado (shape (17,)):
[ 0.75878478 -0.84025675  0.08790717  1.25689885 -1.16550726  1.46611709
 -0.21890065  1.26656662 -0.78666299  0.00507219  0.56333208  0.37435207
  0.51868415 -0.3149266   0.11273953 -2.07770341 -0.20920101]

--- Criando a Rede Neural 2 (nn2) com pesos diferentes ---
Saída da nn2 (antes da cópia): 0.5184

--- Reconstruindo nn2 com os parâmetros da nn1 ---
Parâmetros de nn1 foram copiados para nn2.

--- Verificando os resultados ---
Saída da nn1 original:          -1.83373285
Saída da nn2 após reconstrução: -1.83373285


In [37]:
import time
import numpy as np
import random
from typing import List, Callable, Optional, Tuple
import math
import pygame

# ==============================================================================
# 1. CÓDIGO DA REDE NEURAL (FORNECIDO)
# ==============================================================================

class NN:
    """
    Uma classe para representar uma rede neural simples com uma camada oculta.
    """
    def __init__(self, inputs: int, hidden_units: int, outputs: int, activation: str = 'tanh'):
        """
        Inicializa a rede neural com pesos e biases aleatórios.
        """
        self.inputs = inputs
        self.hidden_units = hidden_units
        self.outputs = outputs
        self.activation_str = activation
        # Inicialização de pesos com He/Xavier para tanh/relu é geralmente melhor
        self.W1 = np.random.randn(hidden_units, inputs) * np.sqrt(2. / inputs)
        self.b1 = np.zeros((1, hidden_units))
        self.W2 = np.random.randn(outputs, hidden_units) * np.sqrt(2. / hidden_units)
        self.b2 = np.zeros((1, outputs))
        self.actv = self._get_activation_function(activation)

    def _get_activation_function(self, name: str) -> Callable[[np.ndarray], np.ndarray]:
        match name:
            case 'sigmoid': return lambda x: 1 / (1 + np.exp(-x))
            case 'tanh': return np.tanh
            case 'relu': return lambda x: np.maximum(0, x)
            case _: raise ValueError(f"Função de ativação '{name}' não suportada.")

    def predict(self, X: np.ndarray) -> np.ndarray:
        if X.ndim == 1: X = X.reshape(1, -1)
        z1 = X @ self.W1.T + self.b1
        a1 = self.actv(z1)
        z2 = a1 @ self.W2.T + self.b2
        # Usar tanh na saída para manter as ações entre -1 e 1
        output = np.tanh(z2) 
        return output

    def __repr__(self):
        return (f"NN(inputs={self.inputs}, hidden_units={self.hidden_units}, "
                f"outputs={self.outputs}, activation='{self.activation_str}')")

# Funções auxiliares para achatamento/reconstrução de parâmetros
def flatten_parameters(nn: NN) -> np.ndarray:
    params = [nn.W1, nn.b1, nn.W2, nn.b2]
    return np.concatenate([p.ravel() for p in params])

def unflatten_parameters(nn: NN, flat_params: np.ndarray):
    s_w1, s_b1, s_w2, s_b2 = nn.W1.size, nn.b1.size, nn.W2.size, nn.b2.size
    total_params = s_w1 + s_b1 + s_w2 + s_b2
    if flat_params.size != total_params:
        raise ValueError(f"Tamanho do array incorreto. Esperado: {total_params}, Recebido: {flat_params.size}")
    pointer = 0
    nn.W1 = flat_params[pointer : pointer + s_w1].reshape(nn.W1.shape); pointer += s_w1
    nn.b1 = flat_params[pointer : pointer + s_b1].reshape(nn.b1.shape); pointer += s_b1
    nn.W2 = flat_params[pointer : pointer + s_w2].reshape(nn.W2.shape); pointer += s_w2
    nn.b2 = flat_params[pointer : pointer + s_b2].reshape(nn.b2.shape)

# ==============================================================================
# 2. CLASSES DO JOGO
# ==============================================================================

# Constantes de configuração
AGENT_SIZE = 12
FRUIT_SIZE = 5
MAX_LIFE = 1000
MAX_STAMINA = 500
LIFE_DECAY_RATE = 30  # Pontos de vida perdidos por segundo (multiplicado por 5 no update)
LIFE_GAIN_ON_EAT = 250
STAMINA_COST_MOVE = 30 # Custo de estamina por segundo de movimento
STAMINA_DEPLETION_SLOW_FACTOR = 0.1 # Fator de lentidão quando estamina = 0
STAMINA_DEPLETION_COOLDOWN_DURATION = 5.0 # Segundos que o agente fica lento após esgotar estamina

FIELD_OF_VIEW_ANGLE = np.deg2rad(30) # 45 graus de campo de visão
VISION_RANGE = 150 # Quão longe o agente pode "ver"

# Cores e transparência para renderização
AGENT_ACTIVE_COLOR = (200, 200, 250) # Azul claro
AGENT_DEAD_COLOR = (80, 80, 80)      # Cinza escuro
FOV_COLOR = (50, 50, 255)            # Azul para FOV
FOV_ALPHA = 40                       # Transparência para FOV
FRUIT_COLOR = (50, 200, 50)          # Verde para fruta
FRUIT_ALPHA = 150                    # Transparência para fruta

class Fruit:
    def __init__(self, world_size: Tuple[int, int]):
        self.world_size = world_size
        self.pos = pygame.math.Vector2(
            random.randint(FRUIT_SIZE, world_size[0] - FRUIT_SIZE),
            random.randint(FRUIT_SIZE, world_size[1] - FRUIT_SIZE)
        )
        # Superfície temporária para desenho com transparência da fruta
        self.transparent_surface = pygame.Surface((FRUIT_SIZE * 2, FRUIT_SIZE * 2), pygame.SRCALPHA)

    def respawn(self):
        self.pos.x = random.randint(FRUIT_SIZE, self.world_size[0] - FRUIT_SIZE)
        self.pos.y = random.randint(FRUIT_SIZE, self.world_size[1] - FRUIT_SIZE)

    def draw(self, surface: pygame.Surface, is_semi_transparent: bool):
        if is_semi_transparent: # Se a renderização semi-transparente está ativa, as frutas também serão
            self.transparent_surface.fill((0, 0, 0, 0)) # Limpa com cor transparente
            local_center = pygame.math.Vector2(FRUIT_SIZE, FRUIT_SIZE)
            pygame.draw.circle(self.transparent_surface, FRUIT_COLOR + (FRUIT_ALPHA,), local_center, FRUIT_SIZE)
            surface.blit(self.transparent_surface, self.pos - local_center)
        else:
            pygame.draw.circle(surface, FRUIT_COLOR, self.pos, FRUIT_SIZE)

class Agent:
    def __init__(self, nn: NN, world_size: Tuple[int, int], use_stamina: bool):
        self.nn = nn
        self.world_size = world_size
        self.use_stamina = use_stamina

        self.pos = pygame.math.Vector2(
            random.randint(0, world_size[0]),
            random.randint(0, world_size[1])
        )
        self.angle = random.uniform(0, 2 * math.pi)
        self.base_speed = 120  # pixels por segundo (velocidade máxima)
        self.rotation_speed = 3.5  # radianos por segundo

        self.life = MAX_LIFE
        self.stamina = MAX_STAMINA if use_stamina else 0

        self.is_alive = True
        self.lifespan = 0.0  # Tempo que o agente viveu

        # NOVAS VARIÁVEIS PARA LÓGICA DE ESTAMINA
        self.is_stamina_depleted = False # Verdadeiro quando a estamina chegou a zero e o cooldown está ativo
        self.stamina_depleted_cooldown = 0.0 # Contador para o cooldown da estamina

        # Superfície temporária para desenho com transparência do agente
        self.agent_surface = pygame.Surface((AGENT_SIZE * 2, AGENT_SIZE * 2), pygame.SRCALPHA)
        # Superfície temporária para desenho com transparência do FOV
        self.fov_surface = pygame.Surface((VISION_RANGE * 2, VISION_RANGE * 2), pygame.SRCALPHA)


    def get_inputs(self, closest_fruit: Optional[Fruit]) -> np.ndarray:
        # Se não há fruta, retorna inputs indicando que não há fruta próxima
        if closest_fruit is None:
            # Ângulo 0, Distância máxima (1.0), Fruta_a_frente 0
            return np.array([0.0, 1.0, 0.0]) 

        vec_to_fruit = closest_fruit.pos - self.pos
        distance = vec_to_fruit.length()
        
        if distance < 1e-5: # Evitar divisão por zero se estiver sobre a fruta
             # Ângulo 0, Distância 0, Fruta_a_frente 1 (muito perto)
             return np.array([0.0, 0.0, 1.0]) 

        # 1. Ângulo da fruta em relação à direção do agente
        angle_to_fruit = math.atan2(vec_to_fruit.y, vec_to_fruit.x)
        relative_angle = angle_to_fruit - self.angle
        # Normalizar ângulo para o intervalo [-pi, pi]
        relative_angle = (relative_angle + math.pi) % (2 * math.pi) - math.pi
        
        # 2. Distância até a fruta (normalizada pela visão)
        normalized_distance = min(1.0, distance / VISION_RANGE)

        # 3. Tem fruta na frente? (dentro do campo de visão e alcance)
        fruit_in_front = 1.0 if (abs(relative_angle) < FIELD_OF_VIEW_ANGLE / 2 and distance < VISION_RANGE) else 0.0

        # Normalizar ângulo relativo para o intervalo [-1, 1]
        normalized_relative_angle = relative_angle / math.pi
        
        return np.array([normalized_relative_angle, normalized_distance, fruit_in_front])

    def update(self, dt: float, action: np.ndarray):
        if not self.is_alive:
            return

        self.lifespan += dt
        
        # Ações baseadas na saída da rede neural [-1, 1]
        rotation_action = action[0]  # Saída 1: Rotação
        move_action = action[1]      # Saída 2: Movimento

        # Girar
        self.angle += rotation_action * self.rotation_speed * dt
        self.angle %= (2 * math.pi) # Manter o ângulo entre 0 e 2*pi

        # --- Lógica de Estamina e Velocidade ---
        current_speed_multiplier = 1.0 # Padrão: velocidade normal
        did_move = False # Flag para saber se o agente tentou mover

        if self.use_stamina:
            if self.is_stamina_depleted:
                # Se a estamina está esgotada, o agente está no cooldown e lento
                self.stamina_depleted_cooldown -= dt
                current_speed_multiplier = STAMINA_DEPLETION_SLOW_FACTOR
                
                if self.stamina_depleted_cooldown <= 0:
                    # Cooldown terminou, resetar o estado de estamina
                    self.is_stamina_depleted = False
                    self.stamina = MAX_STAMINA # Estamina totalmente recarregada
                    self.stamina_depleted_cooldown = 0.0
                    current_speed_multiplier = 1.0 # Volta à velocidade normal
            else:
                # Agente não está no estado de estamina esgotada, gerenciar estamina normalmente
                if move_action > 0.1: # Agente está tentando se mover
                    if self.stamina > 0:
                        self.stamina -= STAMINA_COST_MOVE * dt
                        if self.stamina < 0: self.stamina = 0
                    else:
                        # Estamina acabou, entrar no estado de esgotamento
                        self.is_stamina_depleted = True
                        self.stamina_depleted_cooldown = STAMINA_DEPLETION_COOLDOWN_DURATION
                        current_speed_multiplier = STAMINA_DEPLETION_SLOW_FACTOR # Já fica lento imediatamente
                else: # Agente não está tentando se mover, regenera estamina
                    self.stamina = min(MAX_STAMINA, self.stamina + (STAMINA_COST_MOVE / 2) * dt)
        
        # --- Lógica de Movimento ---
        if move_action > 0.1: # Agente tenta se mover
            did_move = True
            direction = pygame.math.Vector2(math.cos(self.angle), math.sin(self.angle))
            # Aplica o multiplicador de velocidade ao movimento
            self.pos += direction * self.base_speed * current_speed_multiplier * dt * move_action
        
        # Manter o agente dentro dos limites do mundo (wrap around)
        self.pos.x %= self.world_size[0]
        self.pos.y %= self.world_size[1]
        
        # Perder vida ao longo do tempo
        self.life -= LIFE_DECAY_RATE * dt * 5 # Fator 5 para acelerar a simulação
        if self.life <= 0:
            self.is_alive = False
            self.life = 0 # Garante que a vida não fique negativa

    def eat(self):
        self.life = min(MAX_LIFE, self.life + LIFE_GAIN_ON_EAT)

    def draw(self, surface: pygame.Surface, is_semi_transparent: bool, draw_fov: bool):
        current_color = AGENT_ACTIVE_COLOR if self.is_alive else AGENT_DEAD_COLOR

        # Offset para centralizar o desenho local na posição do agente.
        agent_local_center_offset = pygame.math.Vector2(AGENT_SIZE, AGENT_SIZE)
        
        if is_semi_transparent:
            # --- Desenhar FOV (cone de visão) ---
            if draw_fov and self.is_alive: # Desenha FOV apenas se o agente estiver vivo
                self.fov_surface.fill((0, 0, 0, 0)) # Limpa com cor transparente
                
                # Centro da superfície FOV local para o triângulo do FOV
                fov_local_origin = pygame.math.Vector2(VISION_RANGE, VISION_RANGE)
                
                # Pontos do cone do FOV em relação ao fov_local_origin
                p_fov_origin = fov_local_origin # Ponta do cone no centro do agente
                p_fov_left = pygame.math.Vector2(VISION_RANGE, 0).rotate_rad(self.angle - FIELD_OF_VIEW_ANGLE / 2) + fov_local_origin
                p_fov_right = pygame.math.Vector2(VISION_RANGE, 0).rotate_rad(self.angle + FIELD_OF_VIEW_ANGLE / 2) + fov_local_origin
                
                pygame.draw.polygon(self.fov_surface, FOV_COLOR + (FOV_ALPHA,), [p_fov_origin, p_fov_left, p_fov_right])
                # Blit a superfície FOV, deslocada para que seu centro coincida com a posição do agente
                surface.blit(self.fov_surface, self.pos - fov_local_origin)

            # --- Desenhar Agente (triângulo) ---
            self.agent_surface.fill((0, 0, 0, 0)) # Limpa com cor transparente
            
            # Pontos do triângulo relativo ao centro da agent_surface
            local_p1 = pygame.math.Vector2(AGENT_SIZE, 0).rotate_rad(self.angle) + agent_local_center_offset
            local_p2 = pygame.math.Vector2(-AGENT_SIZE/2, -AGENT_SIZE/2).rotate_rad(self.angle) + agent_local_center_offset
            local_p3 = pygame.math.Vector2(-AGENT_SIZE/2, AGENT_SIZE/2).rotate_rad(self.angle) + agent_local_center_offset

            pygame.draw.polygon(self.agent_surface, current_color + (100,), [local_p1, local_p2, local_p3])
            # Blit a superfície do agente, deslocada para que seu centro coincida com a posição do agente
            surface.blit(self.agent_surface, self.pos - agent_local_center_offset)
        else:
            # Desenha o agente diretamente na tela principal (opaco)
            p1 = pygame.math.Vector2(AGENT_SIZE, 0).rotate_rad(self.angle) + self.pos
            p2 = pygame.math.Vector2(-AGENT_SIZE/2, -AGENT_SIZE/2).rotate_rad(self.angle) + self.pos
            p3 = pygame.math.Vector2(-AGENT_SIZE/2, AGENT_SIZE/2).rotate_rad(self.angle) + self.pos
            pygame.draw.polygon(surface, current_color, [p1, p2, p3])

        # --- Renderizar barras de vida e estamina (sempre opacas) ---
        bar_width = AGENT_SIZE * 2
        bar_height = 3
        bar_offset_y = AGENT_SIZE + 2 # Espaço abaixo do agente

        # Posição base para as barras (centralizada abaixo do agente)
        bar_base_x = self.pos.x - AGENT_SIZE
        bar_base_y = self.pos.y + bar_offset_y

        # Barra de Vida
        life_percentage = self.life / MAX_LIFE
        life_bar_color = (0, 200, 0) if self.is_alive else (50, 50, 50) # Verde se vivo, cinza escuro se morto
        
        pygame.draw.rect(surface, (50, 50, 50), (bar_base_x, bar_base_y, bar_width, bar_height)) # Fundo da barra
        pygame.draw.rect(surface, life_bar_color, (bar_base_x, bar_base_y, bar_width * life_percentage, bar_height)) # Preenchimento

        # Barra de Estamina (se ativada)
        if self.use_stamina:
            stamina_percentage = self.stamina / MAX_STAMINA
            stamina_bar_color = (0, 0, 200) if not self.is_stamina_depleted else (100, 100, 255) # Azul normal ou um azul mais claro/indicador de cooldown
            
            bar_base_y += bar_height + 2 # Posição abaixo da barra de vida
            pygame.draw.rect(surface, (50, 50, 50), (bar_base_x, bar_base_y, bar_width, bar_height)) # Fundo da barra
            pygame.draw.rect(surface, stamina_bar_color, (bar_base_x, bar_base_y, bar_width * stamina_percentage, bar_height)) # Preenchimento


class GameWorld:
    def __init__(self, nn: NN, world_size: Tuple[int, int], num_fruits: int, use_stamina: bool):
        self.nn = nn # Guarda a NN original para o retorno (flattened_parameters)
        self.world_size = world_size
        self.agent = Agent(nn, world_size, use_stamina)
        self.fruits = [Fruit(world_size) for _ in range(num_fruits)]
        self._lifespan_recorded = False

    def find_closest_fruit(self) -> Optional[Fruit]:
        if not self.agent.is_alive:
            return None # Agente morto não precisa de fruta

        closest_fruit = None
        min_dist_sq = float('inf')
        for fruit in self.fruits:
            dist_sq = self.agent.pos.distance_squared_to(fruit.pos)
            if dist_sq < min_dist_sq:
                min_dist_sq = dist_sq
                closest_fruit = fruit
        return closest_fruit

    def update(self, dt: float):
        if not self.agent.is_alive:
            return # Não atualiza agentes mortos

        closest_fruit = self.find_closest_fruit()
        
        # Tomada de decisão: Passa a fruta mais próxima (ou None) para obter inputs
        inputs = self.agent.get_inputs(closest_fruit)
        actions = self.agent.nn.predict(inputs)[0] # A predição retorna um array 2D

        # Atualização do estado
        self.agent.update(dt, actions)
        
        # Checar colisão com frutas
        if self.agent.is_alive and closest_fruit is not None:
            if self.agent.pos.distance_to(closest_fruit.pos) < AGENT_SIZE: # Colisão com a fruta mais próxima
                self.agent.eat()
                closest_fruit.respawn()
    
    def draw(self, surface: pygame.Surface, is_semi_transparent: bool, draw_fov: bool):
        # Frutas são desenhadas com transparência se o modo 'is_semi_transparent' estiver ativo
        for fruit in self.fruits:
            fruit.draw(surface, is_semi_transparent=is_semi_transparent)
        
        # Agente (e suas barras) são desenhados com transparência e FOV se solicitado
        self.agent.draw(surface, is_semi_transparent, draw_fov)

# ==============================================================================
# 3. FUNÇÃO PRINCIPAL DA SIMULAÇÃO
# ==============================================================================

def run_game_simulation(
    neural_networks: List[NN],
    world_size: Tuple[int, int] = (800, 600),
    num_fruits: int = 10,
    use_stamina: bool = False,
    render_percentage: float = 0.1,
    skip_frames: int = 0,
    render_fov: bool = False # Nova opção para renderizar o FOV
) -> List[Tuple[np.ndarray, float]]:
    """
    Roda a simulação do jogo com múltiplos agentes. Implementa um loop de jogo
    com fixed timestep para garantir que a lógica da simulação avance de forma
    consistente, desacoplada da taxa de renderização.

    Args:
        neural_networks (List[NN]): Uma lista de redes neurais, uma para cada agente.
        world_size (Tuple[int, int]): O tamanho (largura, altura) do mundo do jogo.
        num_fruits (int): Quantidade de frutas em cada mapa.
        use_stamina (bool): Se os agentes devem ter e usar estamina.
        render_percentage (float): A porcentagem de agentes a serem renderizados (0.0 a 1.0).
        skip_frames (int): Quantidade de quadros a pular entre renderizações.
                           -1 para não renderizar nada (modo headless).
                           0 para renderizar todos os frames.
                           N > 0 para pular N frames (renderiza 1 a cada N+1 frames).
        render_fov (bool): Se o campo de visão do agente deve ser renderizado.

    Returns:
        List[Tuple[np.ndarray, float]]: Uma lista de tuplas contendo
                                        (parâmetros achatados da NN do agente, tempo de vida).
    """
    if skip_frames != -1:
        pygame.init()
        screen = pygame.display.set_mode(world_size)
        pygame.display.set_caption("Simulação de Agentes Inteligentes")
        font = pygame.font.SysFont(None, 24)
    
    clock = pygame.time.Clock()

    print(f"Iniciando simulação com {len(neural_networks)} agentes.")
    
    # Constantes para o fixed timestep
    FIXED_FPS = 60 # A simulação lógica sempre avança como se estivesse a 60 FPS
    FIXED_DT = 1.0 / FIXED_FPS # O passo de tempo fixo para cada atualização do agente

    # Cria um mundo para cada rede neural
    worlds = [GameWorld(nn, world_size, num_fruits, use_stamina) for nn in neural_networks]
    
    # Seleciona quais agentes serão renderizados (primeira seleção)
    num_to_render = int(len(worlds) * render_percentage)
    indices_to_render = random.sample(range(len(worlds)), k=min(len(worlds), num_to_render))
    print(f"Renderizando {len(indices_to_render)} agentes.")
    
    # Lista para armazenar (parâmetros da NN, tempo de vida)
    agent_results = []
    
    running = True
    
    accumulator = 0.0 # Acumula o tempo real passado, para ser consumido em passos de simulação fixos
    logical_simulation_frame_counter = 0 # Conta quantos passos de simulação fixos (frames lógicos) ocorreram
    
    # Inicializa o tempo real para calcular o real_dt
    last_real_time_ms = pygame.time.get_ticks() if skip_frames != -1 else int(time.time() * 1000)

    while running:
        # 1. Calcula o tempo real decorrido desde a última iteração do loop principal.
        # Usa pygame.time.get_ticks() se a renderização estiver ativa, ou time.time() para modo headless.
        current_real_time_ms = pygame.time.get_ticks() if skip_frames != -1 else int(time.time() * 1000)
        real_dt = (current_real_time_ms - last_real_time_ms) / 1000.0
        last_real_time_ms = current_real_time_ms

        # Limita o real_dt para evitar grandes saltos de tempo após pausas ou lags
        if real_dt > 0.1: # Máximo de 100ms por passo de tempo real
            real_dt = 0.1
        accumulator += real_dt

        # 2. Manipulação de eventos Pygame (apenas se a renderização estiver ativa)
        if skip_frames != -1:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                if event.type == pygame.KEYDOWN: # Garante que só reage a teclas pressionadas
                    if event.key == pygame.K_SPACE: # Alternar agentes renderizados
                        alive_world_indices = [i for i, world in enumerate(worlds) if world.agent.is_alive]
                        if len(alive_world_indices) > 0:
                            num_to_sample = min(len(alive_world_indices), num_to_render)
                            indices_to_render = random.sample(alive_world_indices, k=num_to_sample)
                            print(f"Alternando para renderizar {len(indices_to_render)} novos agentes.")
                        else:
                            indices_to_render = []
                            print("Nenhum agente vivo para renderizar.")
                    if event.key == pygame.K_f: # Alternar renderização do FOV
                        render_fov = not render_fov
                        print(f"Renderização do FOV: {'Ativada' if render_fov else 'Desativada'}")

        # 3. Atualizações da simulação (com passo de tempo fixo)
        # Consome o tempo acumulado em passos de simulação fixos (FIXED_DT).
        # A lógica da simulação sempre avança em incrementos de FIXED_DT.
        while accumulator >= FIXED_DT and running: # `and running` para sair se um evento QUIT ocorreu
            for i, world in enumerate(worlds):
                if world.agent.is_alive:
                    world.update(FIXED_DT) # Todos os agentes são atualizados com o FIXED_DT
                elif not world._lifespan_recorded: # Se o agente morreu e seu tempo de vida não foi registrado
                    agent_results.append((flatten_parameters(world.nn), world.agent.lifespan))
                    world._lifespan_recorded = True
            
            accumulator -= FIXED_DT
            logical_simulation_frame_counter += 1

            # Verifica se todos os agentes morreram dentro deste passo de simulação
            active_agents = sum(1 for world in worlds if world.agent.is_alive)
            if active_agents == 0:
                print("Todos os agentes morreram. Fim da simulação.")
                running = False
                break # Sai do loop `while accumulator`

        if not running: # Se a simulação terminou, sai também do loop principal
            break

        # 4. Renderização (se habilitada e for um frame de renderização designado)
        # `skip_frames == -1` significa nenhuma renderização.
        should_render_this_pass = False
        if skip_frames != -1: # Verifica se a renderização não está desabilitada
            # Renderiza se o contador de frames lógicos for maior que 0
            # E se o frame lógico atual se alinha com o intervalo de renderização (skip_frames + 1)
            if logical_simulation_frame_counter > 0 and (logical_simulation_frame_counter % (skip_frames + 1) == 0):
                should_render_this_pass = True

        if should_render_this_pass:
            screen.fill((10, 20, 40)) # Fundo azul escuro
            
            # Desenha apenas os agentes selecionados
            for i in indices_to_render:
                worlds[i].draw(screen, is_semi_transparent=True, draw_fov=render_fov)
            
            # Atualiza a contagem de agentes ativos para exibição
            # (já foi calculada no final do loop `while accumulator`)
            
            # Mostra informações
            # clock.get_fps() agora reflete a taxa de frames *renderizados*
            info_text = (f"Agentes Vivos: {active_agents}/{len(worlds)} | FPS: {clock.get_fps():.1f} | "
                         f"FOV: {'ON' if render_fov else 'OFF'} | Sim Frames: {logical_simulation_frame_counter}")
            text_surface = font.render(info_text, True, (255, 255, 255))
            screen.blit(text_surface, (10, 10))
            
            pygame.display.flip()
            
            # Isso limita a taxa de frames *renderizados* ao FIXED_FPS.
            # É importante que isso ocorra APÓS a renderização.
            clock.tick(FIXED_FPS)
    
    # Finalização
    end_time = time.time()
    print(f"Simulação concluída em {end_time - start_time:.2f} segundos.")

    if skip_frames != -1:
        pygame.quit()
        
    # Adiciona os resultados dos agentes que ainda estavam vivos no final
    for world in worlds:
        if not world._lifespan_recorded:
            agent_results.append((flatten_parameters(world.nn), world.agent.lifespan))

    return agent_results

# ==============================================================================
# 4. BLOCO DE EXEMPLO DE USO
# ==============================================================================

if __name__ == '__main__':
    # --- Parâmetros da Simulação ---
    NUM_AGENTS = 100 # Um número maior para ver a alternância
    USE_STAMINA_OPTION = True
    RENDER_PERCENT = 0.1  # Mostrar 10% dos agentes (10 agentes neste caso)
    SKIP_FRAMES_OPTION = 0 # 0 = renderizar todos os frames, 5 = pular 5 frames, -1 = não renderizar (headless)
    RENDER_FOV_OPTION = True # Renderizar o campo de visão por padrão

    # --- Criação das Redes Neurais ---
    # Cada agente precisa de 3 entradas e 2 saídas
    # Entradas: angulo_relativo, distancia, fruta_a_frente
    # Saídas: girar, andar (valores entre -1 e 1)
    input_neurons = 3
    hidden_neurons = 6
    output_neurons = 2
    
    agent_networks = [
        NN(inputs=input_neurons, hidden_units=hidden_neurons, outputs=output_neurons, activation='tanh')
        for _ in range(NUM_AGENTS)
    ]

    # --- Executar a Simulação ---
    results = run_game_simulation(
        neural_networks=agent_networks,
        use_stamina=USE_STAMINA_OPTION,
        render_percentage=RENDER_PERCENT,
        skip_frames=SKIP_FRAMES_OPTION,
        num_fruits=15,
        render_fov=RENDER_FOV_OPTION
    )

    # --- Mostrar Resultados ---
    if results:
        # Extrai apenas os tempos de vida para estatísticas
        lifespans = [r[1] for r in results] 
        print("\n--- Resultados da Simulação ---")
        print(f"Total de agentes: {len(lifespans)}")
        print(f"Tempo de vida médio: {np.mean(lifespans):.2f} segundos")
        print(f"Melhor tempo de vida: {np.max(lifespans):.2f} segundos")
        print(f"Pior tempo de vida: {np.min(lifespans):.2f} segundos")

        # Exemplo de como acessar os parâmetros de uma NN específica (ex: a melhor)
        best_agent_idx = np.argmax(lifespans)
        best_nn_params = results[best_agent_idx][0]
        print(f"\nParâmetros da NN do agente com melhor tempo de vida (top 5 elementos): {best_nn_params[:5]}...")

Iniciando simulação com 100 agentes.
Renderizando 10 agentes.
Alternando para renderizar 6 novos agentes.
Simulação concluída em 185.91 segundos.

--- Resultados da Simulação ---
Total de agentes: 100
Tempo de vida médio: 6.97 segundos
Melhor tempo de vida: 12.03 segundos
Pior tempo de vida: 6.67 segundos

Parâmetros da NN do agente com melhor tempo de vida (top 5 elementos): [-1.00436242 -0.25707812 -0.51908819 -0.68837995 -1.69229439]...


In [6]:
import time
import numpy as np
from random import randint
from typing import List

INPUTS = 1
HIDDEN_UNITS = 10
OUTPUT = 1
ACTIVATION = 'relu'


class GeneticOptimizationAgent:
    def __init__(self, probability_mutation=0.01, keep_better=False, tamanho_genoma=10, tamanho_pop=1000):
        self.tamanho_genoma = tamanho_genoma
        self.tamanho_pop = tamanho_pop
        self.probability_mutation = probability_mutation

        self.population = [self.gerar_individuo_aleatoriamente() for i in range(self.tamanho_pop)]

        self.keep_better = keep_better
    
    def gerar_individuo_aleatoriamente(self):
        return np.random.normal(0, 1, self.tamanho_genoma)

    
    def fitness(self):
        return run_game_simulation(
            neural_networks=self.population,
            use_stamina=USE_STAMINA_OPTION,
            render_percentage=RENDER_PERCENT,
            skip_frames=SKIP_FRAMES_OPTION,
            num_fruits=5,
            render_fov=RENDER_FOV_OPTION
        )

    def mutate(self, individual: np.array) -> np.array:
        if len(individual) != self.tamanho_genoma:
            raise Exception("Indivíduo de tamanho errado")
        
        mutated = individual.copy()
        for i in range(len(mutated)):
            if np.random.rand() < self.probability_mutation:
                mutated[i] += np.random.normal(0, 1)

        return mutated
    
    @staticmethod
    def crossover(ind1: np.array, ind2: np.array) -> List[np.array]:
        if len(ind1) != len(ind2):
            raise Exception("Indivíduos de tamanhos diferentes")
        
        tamanho_genoma = len(ind1)
        
        cross_point = np.random.randint(1, tamanho_genoma - 1)

        # Use np.concatenate to join numpy array slices
        child1 = np.concatenate((ind1[:cross_point], ind2[cross_point:]))
        child2 = np.concatenate((ind2[:cross_point], ind1[cross_point:]))

        return [child1, child2]
    
    def selection(self):
        # Implementando torneiro

        new_population = []

        MSE = 0

        fitness_tuple = self.fitness()

        MSE = -max([x[1] for x in fitness_tuple])

        for i in range(self.tamanho_pop//2):
            ind1_index = randint(0, self.tamanho_pop - 1)
            ind2_index = randint(0, self.tamanho_pop - 1)

            pai = fitness_tuple[ind1_index][0] if fitness_tuple[ind1_index][1] > fitness_tuple[ind2_index][1] else fitness_tuple[ind2_index][0]

            ind1_index = randint(0, self.tamanho_pop - 1)
            ind2_index = randint(0, self.tamanho_pop - 1)

            mae = fitness_tuple[ind1_index][0] if fitness_tuple[ind1_index][1] > fitness_tuple[ind2_index][1] else fitness_tuple[ind2_index][0]

            children = self.crossover(pai, mae)

            # Mutação
            children = [self.mutate(child) for child in children]
            new_population.extend(children)

        if self.keep_better:
            best_individual = max(fitness_tuple, key=lambda x: x[1])[0]
            new_population[0] = best_individual
        

        self.population = new_population

        return best_individual, MSE


#### Testando crossover

In [7]:
"""
ind1 = gerar_individuo_aleatoriamente()
ind2 = gerar_individuo_aleatoriamente()

print(f'Pai: \t\t{ind1}, valor: {genoma_para_valor(ind1)}')
print(f'Mãe: \t\t{ind2}, valor: {genoma_para_valor(ind2)}')

child1, child2 = GeneticOptimization.crossover(ind1, ind2)

print(f'Filho1: \t{child1}, valor: {genoma_para_valor(child1)}')
print(f'Filho2: \t{child2}, valor: {genoma_para_valor(child2)}')
"""

"\nind1 = gerar_individuo_aleatoriamente()\nind2 = gerar_individuo_aleatoriamente()\n\nprint(f'Pai: \t\t{ind1}, valor: {genoma_para_valor(ind1)}')\nprint(f'Mãe: \t\t{ind2}, valor: {genoma_para_valor(ind2)}')\n\nchild1, child2 = GeneticOptimization.crossover(ind1, ind2)\n\nprint(f'Filho1: \t{child1}, valor: {genoma_para_valor(child1)}')\nprint(f'Filho2: \t{child2}, valor: {genoma_para_valor(child2)}')\n"

In [27]:
import time
import numpy as np
from random import randint
from typing import List

INPUTS = 1
HIDDEN_UNITS = 10
OUTPUT = 1


class GeneticOptimizationAgent:
    def __init__(self, probability_mutation=0.01, keep_better=False, input_neurons=3, hidden_neurons=5, activation_neruons=3, tamanho_pop=1000, activation='sigmoid'):
        self.tamanho_genoma = (input_neurons * hidden_neurons) + hidden_neurons + (hidden_neurons * activation_neruons) + activation_neruons
        self.tamanho_pop = tamanho_pop
        self.input_neurons = input_neurons
        self.hidden_neurons = hidden_neurons
        self.activation_neruons = activation_neruons
        self.activation_function = activation

        self.probability_mutation = probability_mutation

        self.population = [self.gerar_individuo_aleatoriamente() for i in range(self.tamanho_pop)]

        self.keep_better = keep_better
    
    def gerar_individuo_aleatoriamente(self):
        return np.random.normal(0, 1, self.tamanho_genoma)

    
    def fitness(self, skip_frames=5):

        nns = [NN(inputs=self.input_neurons, hidden_units=self.hidden_neurons, outputs=self.activation_neruons, activation=self.activation_function) for _ in range(self.tamanho_pop)]
        for i in range(self.tamanho_pop):
            unflatten_parameters(nns[i], self.population[i])

        return run_game_simulation(
            neural_networks=nns,
            use_stamina=USE_STAMINA_OPTION,
            render_percentage=0.01,
            skip_frames=skip_frames,
            num_fruits=15,
            render_fov=RENDER_FOV_OPTION
        )

    def mutate(self, individual: np.array) -> np.array:
        if len(individual) != self.tamanho_genoma:
            raise Exception("Indivíduo de tamanho errado")
        
        mutated = individual.copy()
        for i in range(len(mutated)):
            if np.random.rand() < self.probability_mutation:
                mutated[i] += np.random.normal(0, 1)

        return mutated
    
    @staticmethod
    def crossover(ind1: np.array, ind2: np.array) -> List[np.array]:
        if len(ind1) != len(ind2):
            raise Exception("Indivíduos de tamanhos diferentes")
        
        tamanho_genoma = len(ind1)
        
        cross_point = np.random.randint(1, tamanho_genoma - 1)

        # Use np.concatenate to join numpy array slices
        child1 = np.concatenate((ind1[:cross_point], ind2[cross_point:]))
        child2 = np.concatenate((ind2[:cross_point], ind1[cross_point:]))

        return [child1, child2]
    
    def selection(self, skip_frames=5):
        # Implementando torneiro

        new_population = []

        MSE = 0

        fitness_tuple = self.fitness(skip_frames=skip_frames)

        MSE = max([x[1] for x in fitness_tuple])

        for i in range(self.tamanho_pop//2):
            ind1_index = randint(0, self.tamanho_pop - 1)
            ind2_index = randint(0, self.tamanho_pop - 1)

            pai = fitness_tuple[ind1_index][0] if fitness_tuple[ind1_index][1] > fitness_tuple[ind2_index][1] else fitness_tuple[ind2_index][0]

            ind1_index = randint(0, self.tamanho_pop - 1)
            ind2_index = randint(0, self.tamanho_pop - 1)

            mae = fitness_tuple[ind1_index][0] if fitness_tuple[ind1_index][1] > fitness_tuple[ind2_index][1] else fitness_tuple[ind2_index][0]

            children = self.crossover(pai, mae)

            # Mutação
            children = [self.mutate(child) for child in children]
            new_population.extend(children)

        if self.keep_better:
            best_individual = max(fitness_tuple, key=lambda x: x[1])[0]
            new_population[0] = best_individual
        

        self.population = new_population

        return best_individual, MSE


In [ ]:

# Saídas: girar, andar
input_neurons = 3
hidden_neurons = 6
output_neurons = 2

optimizator = GeneticOptimizationAgent(probability_mutation=0.05,keep_better=True, input_neurons=input_neurons, hidden_neurons=hidden_neurons, activation_neruons=output_neurons, tamanho_pop=1000, activation='tanh')


In [40]:
import matplotlib.pyplot as plt 

epochs = 1000

MSE = np.zeros((epochs))  # Lista para armazenar o MSE de cada é

for generation in range(epochs):
    start_time = time.time()
    best_individual, CMSE = optimizator.selection(skip_frames=-1)
    print(f"Geração {generation}: Melhor indivíduo: {best_individual}, Fitness: {CMSE:.6f}, Tempo: {time.time() - start_time:.2f}s")
    MSE[generation] = CMSE 
    

plt.plot(range(epochs), MSE)
plt.xlabel('Geração')
plt.ylabel('MSE do Melhor Indivíduo')
plt.title('Evolução do MSE ao Longo das Gerações')

Iniciando simulação com 1000 agentes.
Renderizando 10 agentes.
Todos os agentes morreram. Fim da simulação.
Simulação concluída em 137.23 segundos.
Geração 0: Melhor indivíduo: [ 5.71553671  1.02250121  6.77570395  4.3850715   1.53078236 -2.45121728
  0.96414029  2.27574364 -2.56788398  6.50257376 -0.3881282   2.2703629
 -2.27787885  0.5879981  -1.48881807 -5.38521627  0.12343052 -6.4062561
 -0.34216214 -0.64305361 -1.64058351 -0.12486857  0.68474781  0.03255587
  3.63682743 -0.98512101  0.70041827  3.19334237 -2.96671251 -4.72379993
 -1.97972895 -0.85787939  0.05818317 -0.51901337  2.39246669  1.13028818
  0.7172406   9.5494716 ], Fitness: 137.050000, Tempo: 137.26s
Iniciando simulação com 1000 agentes.
Renderizando 10 agentes.
Todos os agentes morreram. Fim da simulação.
Simulação concluída em 176.22 segundos.
Geração 1: Melhor indivíduo: [ 2.85338865 -3.98240624  3.53999527  2.99954479 -0.49279259 -6.09995289
  2.59606574 -3.54524034  0.93707441  4.68444834 -1.21641133  2.17677638
 

KeyboardInterrupt: 

In [42]:
optimizator.fitness(skip_frames=0)



Iniciando simulação com 1000 agentes.
Renderizando 10 agentes.
Alternando para renderizar 10 novos agentes.
Alternando para renderizar 10 novos agentes.
Alternando para renderizar 10 novos agentes.
Simulação concluída em 123.92 segundos.


[(array([ 0.50954531,  0.13599417,  2.52336718, -3.4468121 , -1.33973355,
          2.37664972, -1.34494314, -0.90747082,  2.63814679,  3.19176592,
          0.07148849,  1.50327352, -2.84611829, -1.94686617, -3.99793666,
         -6.69311505,  1.29011798, -2.5826178 ,  0.30175506,  1.44124697,
          0.71374799,  1.48876201,  0.04223647,  0.04907206,  2.34046265,
         -0.72291133,  3.06741254,  4.23923439, -3.50974029, -4.41730361,
         -1.97375656, -2.4781791 ,  0.11621746, -2.9416644 ,  1.65730895,
          2.00506209,  3.67097636,  5.76603824]),
  6.666666666666648),
 (array([-2.66369449, -2.00149542,  3.80151393, -1.32023543,  1.94268125,
          3.15455452,  1.5084313 ,  2.12896746, -1.41580549,  4.32917318,
          1.39584211,  1.37027053, -3.34237992, -1.16370195, -3.96719876,
         -7.70764445, -0.54134988, -5.37343836, -3.60400754, -0.18507313,
          0.64875901,  1.58972197,  0.06483196, -1.67461256,  1.97857148,
         -1.05208875,  2.0250258 ,  3.30